In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf /content/spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


In [ ]:
!curl -L http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz -o data.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2402k  100 2402k    0     0  1469k      0  0:00:01  0:00:01 --:--:-- 1469k


In [ ]:
# %fs ls "file:/databricks/driver"

In [ ]:
df = spark.read.json('data.json.gz')
df.show()

NameError: ignored

In [ ]:
# only select the necessary rows
df = df.select('asin','overall','reviewText','reviewerID')
# Convert unique strings to integer using StringIndexer
asin_indexer = StringIndexer(inputCol="asin", outputCol="itemId")
reviewerID_indexer = StringIndexer(inputCol="reviewerID", outputCol="userId")

df = asin_indexer.fit(df).transform(df)
df = reviewerID_indexer.fit(df).transform(df)

In [ ]:
df = df.select('userId','itemId','reviewText','overall')
df = df.withColumnRenamed('overall', 'rating')

df.show()

+------+------+--------------------+------+
|userId|itemId|          reviewText|rating|
+------+------+--------------------+------+
|  66.0| 703.0|Not much to write...|   5.0|
| 266.0| 703.0|The product does ...|   5.0|
| 395.0| 703.0|The primary job o...|   5.0|
|1048.0| 703.0|Nice windscreen p...|   5.0|
|1311.0| 703.0|This pop filter i...|   5.0|
|  51.0| 562.0|So good that I bo...|   5.0|
| 290.0| 562.0|I have used monst...|   5.0|
| 374.0| 562.0|I now use this ca...|   3.0|
|  13.0| 562.0|Perfect for my Ep...|   5.0|
| 183.0| 562.0|Monster makes the...|   5.0|
|   4.0| 562.0|Monster makes a w...|   5.0|
| 488.0| 704.0|I got it to have ...|   4.0|
| 699.0| 704.0|If you are not us...|   3.0|
|  49.0| 704.0|I love it, I used...|   5.0|
| 594.0| 704.0|I bought this to ...|   5.0|
| 317.0| 704.0|I bought this to ...|   2.0|
| 104.0| 455.0|This Fender cable...|   4.0|
| 250.0| 455.0|wanted it just on...|   5.0|
|   3.0| 455.0|I've been using t...|   5.0|
|  29.0| 455.0|Fender cords look

In [ ]:
df = df.withColumn('userId', col('userId').cast('integer')).\
        withColumn('itemId', col('itemId').cast('integer')).\
        withColumn('rating', col('rating').cast('float')).\
        drop('reviewText')

df.show()

+------+------+------+
|userId|itemId|rating|
+------+------+------+
|    66|   703|   5.0|
|   266|   703|   5.0|
|   395|   703|   5.0|
|  1048|   703|   5.0|
|  1311|   703|   5.0|
|    51|   562|   5.0|
|   290|   562|   5.0|
|   374|   562|   3.0|
|    13|   562|   5.0|
|   183|   562|   5.0|
|     4|   562|   5.0|
|   488|   704|   4.0|
|   699|   704|   3.0|
|    49|   704|   5.0|
|   594|   704|   5.0|
|   317|   704|   2.0|
|   104|   455|   4.0|
|   250|   455|   5.0|
|     3|   455|   5.0|
|    29|   455|   5.0|
+------+------+------+
only showing top 20 rows



In [ ]:
(train, test) = df.randomSplit([0.8, 0.2], seed = 1234)
als = ALS(maxIter=20, regParam=0.05, rank=100,userCol="userId", itemCol="itemId", ratingCol="rating",coldStartStrategy="drop")
model = als.fit(train)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.1278291834229006
